In [200]:
from xml.dom import minidom 
import os  
import pandas as pd
from bs4 import BeautifulSoup
import csv, sys, os
from lxml import etree
import xml.etree.ElementTree as ET

In [201]:
#get the dataset https://developer.salesforce.com/docs/metadata-coverage/51
df = pd.read_csv("metadataCoverage.csv")
df.head()

,Metadata Type,Metadata API,Source Tracking,Unlocked Packaging Second generation Unlocked Packaging,Managed Packaging Second generation Managed Packaging,"Classic Packaging First generation Packaging, and covers both Managed and Unmanaged through footnotes in the cells",Change Sets,Apex Metadata API,Known Issues
0,AccessControlPolicy,Exposed in Metadata API,Exposed in Source Tracking,Requires package without a namespace. 1,NaN,NaN,NaN,NaN,NaN
1,AccountForecastSettings,Exposed in Metadata API,Exposed in Source Tracking,Exposed in Unlocked Packaging,NaN,NaN,NaN,NaN,NaN
2,AccountInsightsSettings,Exposed in Metadata API,Exposed in Source Tracking,NaN,NaN,NaN,NaN,NaN,NaN
3,AccountIntelligenceSettings,Exposed in Metadata API,Exposed in Source Tracking,NaN,NaN,NaN,NaN,NaN,NaN
4,AccountRelationshipShareRule,Exposed in Metadata API,Exposed in Source Tracking,Exposed in Unlocked Packaging,Exposed in Managed Packaging,Exposed in Classic Packaging,Exposed in Change Sets,NaN,NaN


In [202]:
#drop the irrelevant columns - only need metadata type, metadata API, and unlocked packaging columns 

cols = [2, 4, 5, 6, 7, 8]
df.drop(df.columns[cols], axis = 1, inplace= True)
df.head()

,Metadata Type,Metadata API,Unlocked Packaging Second generation Unlocked Packaging
0,AccessControlPolicy,Exposed in Metadata API,Requires package without a namespace. 1
1,AccountForecastSettings,Exposed in Metadata API,Exposed in Unlocked Packaging
2,AccountInsightsSettings,Exposed in Metadata API,NaN
3,AccountIntelligenceSettings,Exposed in Metadata API,NaN
4,AccountRelationshipShareRule,Exposed in Metadata API,Exposed in Unlocked Packaging


In [203]:
#rename column names
df.columns = ['metadataType', 'metadataAPI', 'unlockedPackaging']
df.head()

,metadataType,metadataAPI,unlockedPackaging
0,AccessControlPolicy,Exposed in Metadata API,Requires package without a namespace. 1
1,AccountForecastSettings,Exposed in Metadata API,Exposed in Unlocked Packaging
2,AccountInsightsSettings,Exposed in Metadata API,NaN
3,AccountIntelligenceSettings,Exposed in Metadata API,NaN
4,AccountRelationshipShareRule,Exposed in Metadata API,Exposed in Unlocked Packaging


In [204]:
#save the csv file
df.to_csv('metadataCoverage_clean.csv')

In [205]:
#create three different datasets - 
# 1. where metadata type is available in metadata API
# 2. where metadata type is available in unlocked packaging
# 3. where metadata type is not available in unlocked packaging

# 1. where metadata type is available in metadata API

metadataAPI_available_data = df.drop(df.columns[2], axis = 1) #drop unlocked packaging
metadataAPI_available_data = metadataAPI_available_data.dropna()

# 2. where metadata type is available in unlocked packaging

unlockedPackaging_data = df.drop(df.columns[1], axis = 1) #drop metadataAPI
unlockedPackaging_data_available = unlockedPackaging_data.dropna()

# 3. where metadata type is not available in unlocked packaging

unlockedPackaging_data_notAvailable = unlockedPackaging_data[unlockedPackaging_data['unlockedPackaging'].isnull()]
unlockedPackaging_data_notAvailable.columns = ['metadataType', 'unlockedPackaging_nan']


In [206]:
def createXML(data):
    package = ET.Element('Package')
    package.set('xmlns', 'http://soap.sforce.com/2006/04/metadata')
    for index, row in data.iterrows():
        types = ET.SubElement(package, 'types')
        members = ET.SubElement(types, 'members')
        members.text = '*'
        name = ET.SubElement(types, 'name')
        name.text = row['metadataType'] 

    mydata = '<?xml version="1.0" encoding="UTF-8"?>\n' + ET.tostring(package, encoding="unicode")
    
    xmlfilename = data.columns[1] + ".xml"

    with open(xmlfilename, "w") as f: 
        f.write(BeautifulSoup(mydata, "xml").prettify())
    


In [207]:
createXML(metadataAPI_available_data)
createXML(unlockedPackaging_data_available)
createXML(unlockedPackaging_data_notAvailable)
